In [ ]:
! pip install -q transformers -U  #Hugginface transformer
import numpy as np
import pandas as pd
import tensorflow as tf
import torch
from transformers import FlaubertModel, FlaubertTokenizer #Tokenizer for french
from scipy.spatial.distance import cosine #to compute similarity
from sklearn.manifold import TSNE #To make visualisation
import matplotlib.pyplot as plt #ploting
import plotly.express as px #Ploting
from transformers import T5Tokenizer, T5ForConditionalGeneration, T5Config

In [ ]:
% pip install sentencepiece
model = T5ForConditionalGeneration.from_pretrained('t5-large')
tokenizer = T5Tokenizer.from_pretrained('t5-large')
device = torch.device('cpu')

Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

In [ ]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

In [ ]:
df = pd.read_csv('/content/lest_republicain_summaries.csv',encoding='utf-8')

In [ ]:
text = df[['text','summary']]

In [ ]:
print(df['summary'][4])
print(df['text'][4])

Selon une étude américaine publiée mercredi, les personnes d'abord vaccinées avec Johnson & Johnson, on vu leurs niveaux d'anticorps multipliés par 4 après une dose de rappel du même vaccin, par 35 après un rappel de Pfizer, et par 76 après un rappel de Moderna.
Les personnes ayant reçu le vaccin contre le Covid-19 de Johnson & Johnson pourraient avoir intérêt à recevoir une dose de rappel d'un vaccin différent, à ARN messager, selon les résultats préliminaires d'une étude américaine publiée mercredi. Cette étude, financée par les Instituts nationaux de santé (NIH), était très attendue aux Etats-Unis car elle porte sur la possibilité de "mélanger" les vaccins -  c'est-à-dire d'utiliser un vaccin différent pour la dose de rappel que pour la série initiale - ce qui n'est pas autorisé pour le moment dans le pays. L'étude a été menée sur 458 adultes vaccinés avec l'un des trois remèdes autorisés aux Etats-Unis (Pfizer, Moderna ou Johnson & Johnson), depuis au moins 12 semaines. Ces trois g

In [ ]:
df.head()

,Unnamed: 0,date,summary,text,title,topic,url,relevance
0,0,2021/10/14,Suivez avec nous l'évolution de la pandémie de...,20h47 : 6523 patients hospitalisés et 38 nouve...,En direct. Covid-19 : 6523 patients hospitalis...,NaN,https://www.estrepublicain.fr/sante/2021/10/14...,NaN
1,1,2020/09/21,"Vaccination, taux d’incidence, nombre de patie...","Au jeudi 14 octobre 2021, 393 patients sont ho...",Infographies. Covid-19 : 1 décès supplémentair...,NaN,https://www.estrepublicain.fr/sante/2020/09/21...,NaN
2,3,2021/10/14,"Le laboratoire AstraZeneca, qui propose déjà u...",Le régulateur européen a indiqué jeudi avoir c...,Covid-19. Le régulateur européen évalue Evushe...,NaN,https://www.estrepublicain.fr/sante/2021/10/14...,NaN
3,4,2021/10/14,L'OMS a dévoilé mercredi une nouvelle équipe d...,Ils sont 26 membres proposés par l'OMS pour fa...,Covid-19. Qui sont les nouveaux experts chargé...,NaN,https://www.estrepublicain.fr/sante/2021/10/14...,NaN
4,5,2021/10/14,"Selon une étude américaine publiée mercredi, l...",Les personnes ayant reçu le vaccin contre le C...,Covid-19. Un rappel de Moderna ou Pfizer march...,NaN,https://www.estrepublicain.fr/sante/2021/10/14...,NaN


In [ ]:
# Creating a custom dataset for reading the dataframe and loading it into the dataloader to pass it to the neural network at a later stage for finetuning the model and to prepare it for predictions
from torch.utils.data import Dataset, DataLoader
class CustomDataset(Dataset):

    def __init__(self, dataframe, tokenizer, source_len, summ_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.source_len = source_len
        self.summ_len = summ_len
        self.summary = self.data.summary
        self.text = self.data.text

    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        text = str(self.text[index])
        text = ' '.join(text.split())

        summary = str(self.summary[index])
        summary = ' '.join(summary.split())

        source = self.tokenizer.batch_encode_plus([ text ], max_length= self.source_len, pad_to_max_length=True,return_tensors='pt')
        target = self.tokenizer.batch_encode_plus([ summary ], max_length= self.summ_len, pad_to_max_length=True,return_tensors='pt')

        source_ids = source['input_ids'].squeeze()
        source_mask = source['attention_mask'].squeeze()
        target_ids = target['input_ids'].squeeze()
        target_mask = target['attention_mask'].squeeze()

        return {
            'source_ids': source_ids.to(dtype=torch.long), 
            'source_mask': source_mask.to(dtype=torch.long), 
            'target_ids': target_ids.to(dtype=torch.long),
            'target_ids_y': target_ids.to(dtype=torch.long)
        }

In [ ]:
# Creating the training function. This will be called in the main function. It is run depending on the epoch value.
# The model is put into train mode and then we wnumerate over the training loader and passed to the defined network 

def train(epoch, tokenizer, model, device, loader, optimizer):
    model.train()
    for _,data in enumerate(loader, 0):
        y = data['target_ids'].to(device, dtype = torch.long)
        y_ids = y[:, :-1].contiguous()
        labels = y[:, 1:].clone().detach()
        labels[y[:, 1:] == tokenizer.pad_token_id] = -100
        ids = data['source_ids'].to(device, dtype = torch.long)
        mask = data['source_mask'].to(device, dtype = torch.long)

        outputs = model(input_ids = ids, attention_mask = mask, decoder_input_ids=y_ids, labels=labels)
        loss = outputs[0]
        
        if _%500==0:
            print(f'Epoch: {epoch}, Loss:  {loss.item()}')
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # xm.optimizer_step(optimizer)
        # xm.mark_step()

In [ ]:
def validate(epoch, tokenizer, model, device, loader):
    model.eval()
    predictions = []
    actuals = []
    with torch.no_grad():
        for _, data in enumerate(loader, 0):
            y = data['target_ids'].to(device, dtype = torch.long)
            ids = data['source_ids'].to(device, dtype = torch.long)
            mask = data['source_mask'].to(device, dtype = torch.long)

            generated_ids = model.generate(
                input_ids = ids,
                attention_mask = mask, 
                max_length=150, 
                num_beams=2,
                repetition_penalty=2.5, 
                length_penalty=1.0, 
                early_stopping=True
                )
            preds = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in generated_ids]
            target = [tokenizer.decode(t, skip_special_tokens=True, clean_up_tokenization_spaces=True)for t in y]
            if _%100==0:
                print(f'Completed {_}')

            predictions.extend(preds)
            actuals.extend(target)
    return predictions, actuals

In [ ]:
def main():

    TRAIN_BATCH_SIZE = 2    # input batch size for training (default: 64)
    VALID_BATCH_SIZE = 2    # input batch size for testing (default: 1000)
    TRAIN_EPOCHS = 2        # number of epochs to train (default: 10)
    VAL_EPOCHS = 1 
    LEARNING_RATE = 1e-4    # learning rate (default: 0.01)
    SEED = 42               # random seed (default: 42)
    MAX_LEN = 512
    SUMMARY_LEN = 150 

    # Set random seeds and deterministic pytorch for reproducibility
    torch.manual_seed(SEED) # pytorch random seed
    np.random.seed(SEED) # numpy random seed
    torch.backends.cudnn.deterministic = True

    # tokenzier for encoding the text
    tokenizer = T5Tokenizer.from_pretrained("t5-base")
    

    # Importing and Pre-Processing the domain data
    # Selecting the needed columns only. 
    # Adding the summarzie text in front of the text. This is to format the dataset similar to how T5 model was trained for summarization task. 
    df = pd.read_csv('/content/lest_republicain_summaries.csv',encoding='utf-8')
    df = df[['summary','text']]
    df.text = 'summarize: ' + df.text
    print(df.head())

    
    # Creation of Dataset and Dataloader
    # Defining the train size. So 80% of the data will be used for training and the rest will be used for validation. 
    train_size = 0.8
    train_dataset=df.sample(frac=train_size, random_state = SEED).reset_index(drop=True)
    val_dataset=df.drop(train_dataset.index).reset_index(drop=True)

    print("FULL Dataset: {}".format(df.shape))
    print("TRAIN Dataset: {}".format(train_dataset.shape))
    print("TEST Dataset: {}".format(val_dataset.shape))


    # Creating the Training and Validation dataset for further creation of Dataloader
    training_set = CustomDataset(train_dataset, tokenizer, MAX_LEN, SUMMARY_LEN)
    val_set = CustomDataset(val_dataset, tokenizer, MAX_LEN, SUMMARY_LEN)

    # Defining the parameters for creation of dataloaders
    train_params = {
        'batch_size': TRAIN_BATCH_SIZE,
        'shuffle': True,
        'num_workers': 0
        }

    val_params = {
        'batch_size': VALID_BATCH_SIZE,
        'shuffle': False,
        'num_workers': 0
        }

    # Creation of Dataloaders for testing and validation. This will be used down for training and validation stage for the model.
    training_loader = DataLoader(training_set, **train_params)
    val_loader = DataLoader(val_set, **val_params)


    
    # Defining the model. We are using t5-base model and added a Language model layer on top for generation of Summary. 
    # Further this model is sent to device (GPU/TPU) for using the hardware.
    model = T5ForConditionalGeneration.from_pretrained("t5-base")
    model = model.to(device)

    # Defining the optimizer that will be used to tune the weights of the network in the training session. 
    optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

    # Training loop
    print('Initiating Fine-Tuning for the model on our dataset')

    for epoch in range(TRAIN_EPOCHS):
        train(epoch, tokenizer, model, device, training_loader, optimizer)


    # Validation loop and saving the resulting file with predictions and acutals in a dataframe.
    # Saving the dataframe as predictions.csv
    print('Now generating summaries on our fine tuned model for the validation dataset and saving it in a dataframe')
    for epoch in range(VAL_EPOCHS):
        predictions, actuals = validate(epoch, tokenizer, model, device, val_loader)
        final_df = pd.DataFrame({'Generated Text':predictions,'Actual Text':actuals})
        final_df.to_csv('predictions.csv')
        print('Output Files generated for review')

if __name__ == '__main__':
    main()

                                             summary                                               text
0  Suivez avec nous l'évolution de la pandémie de...  summarize: 20h47 : 6523 patients hospitalisés ...
1  Vaccination, taux d’incidence, nombre de patie...  summarize: Au jeudi 14 octobre 2021, 393 patie...
2  Le laboratoire AstraZeneca, qui propose déjà u...  summarize: Le régulateur européen a indiqué je...
3  L'OMS a dévoilé mercredi une nouvelle équipe d...  summarize: Ils sont 26 membres proposés par l'...
4  Selon une étude américaine publiée mercredi, l...  summarize: Les personnes ayant reçu le vaccin ...
FULL Dataset: (1762, 2)
TRAIN Dataset: (1410, 2)
TEST Dataset: (352, 2)


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Initiating Fine-Tuning for the model on our dataset


/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2218: FutureWarning:

The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).



In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
import pandas as pd


In [ ]:
exclure_mots = ['d', 'du', 'de', 'la', 'des', 'le', 'et', 'est', 'elle', 'une',
                'en', 'que', 'aux', 'qui', 'ces', 'les', 'dans', 'sur', 'l', 'un', 'pour', 'par', 'il', 
                'ou', 'à', 'ce', 'a', 'sont', 'cas',
                'plus', 'leur', 'se', 's', 'vous', 'au', 'c', 'aussi', 'toutes', 'autre', 'comme']

In [ ]:
plt.figure(figsize=(8,10))
word_cloud = WordCloud(background_color = 'white', stopwords = exclure_mots,
                       ).generate(df['text'][7])
plt.imshow(word_cloud)
plt.axis("off")
plt.show()

In [ ]:
def data_prep(text):
  max_source_length = 512
  max_target_length = 128
  input_sequences = text
  encoding = tokenizer.batch_encode_plus(input_sequences, max_length= source_len,
                                         pad_to_max_length=True,return_tensors='pt')

In [ ]:
text = "Un nuage de fumée juste après l’explosion, le 1er juin 2019.\
Une déflagration dans une importante usine d’explosifs du centre de la Russie a fait au moins 79 blessés samedi 1er juin.\
 L’explosion a eu lieu dans l’usine Kristall à Dzerzhinsk, une ville située à environ 400 kilomètres à l’est de Moscou,\
  dans la région de Nijni-Novgorod. « Il y a eu une explosion technique dans l’un des ateliers,\
   suivie d’un incendie qui s’est propagé sur une centaine de mètres carrés », a expliqué un porte-parole des services d’urgence. \
   Des images circulant sur les réseaux sociaux montraient un énorme nuage de fumée après l’explosion. \
   Cinq bâtiments de l’usine et près de 180 bâtiments résidentiels ont été endommagés par l’explosion,\
    selon les autorités municipales. Une enquête pour de potentielles violations des normes de sécurité a été ouverte.\
     Fragments de shrapnel Les blessés ont été soignés après avoir été atteints par des fragments issus de l’explosion, \
     a précisé une porte-parole des autorités sanitaires citée par Interfax. « Nous parlons de blessures par shrapnel d’une gravité moyenne et modérée »,\
     a-t-elle précisé. Selon des représentants de Kristall, cinq personnes travaillaient dans la zone où s’est produite l’explosion. \
     Elles ont pu être évacuées en sécurité. Les pompiers locaux ont rapporté n’avoir aucune information sur des personnes qui se trouveraient encore dans l’usine."

In [ ]:
import json

In [ ]:
#f = open('/content/actu_preliminary.json')
#text = json.load(f)

In [ ]:
transformers.PreTrainedTokenizerBase.__call__